# 使用 Milvus 和 DeepSeek 构建 民法典 RAG


In [1]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
os.environ['HUGGINGFACE_CO_URL_HOME'] = 'https://hf-mirror.com'

In [2]:
from glob import glob

text_lines = []

with open("mfd.md", "r") as file:
    file_text = file.read()

text_lines += file_text.split("\n\n")

In [3]:
len(text_lines)

417

### 准备 LLM 和 Embedding 模型

In [4]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [5]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

/data/miniconda/install/envs/deepseek/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo_mfd1.db")

collection_name = "mfd_rag_collection1"

In [7]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [9]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)

In [10]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim, 
    metric_type="COSINE",  # 内积距离#余弦相似度
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

In [11]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating mfd embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "txt": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating mfd embeddings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 417/417 [00:00<00:00, 1510384.08it/s]


{'insert_count': 417, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

In [15]:
question = "登记机构应履行那些职责?"

In [16]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "COSINE", "params": {}},  # 内积距离
    output_fields=["txt"],  # 返回 text 字段
)

In [17]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["txt"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "**\u7b2c\u4e09\u767e\u516b\u5341\u516d\u6761** \u5c45\u4f4f\u6743\u4eba\u6709\u6743\u5bf9\u5c45\u4f4f\u6743\u7684\u4f4f\u5b85\u8fdb\u884c\u7ef4\u4fee\u3001\u517b\u62a4\uff0c\u4f46\u4e0d\u5f97\u6539\u53d8\u4f4f\u5b85\u7684\u7528\u9014\u3002\n\u5c45\u4f4f\u6743\u4eba\u5bf9\u5c45\u4f4f\u6743\u7684\u4f4f\u5b85\u4eab\u6709\u5360\u6709\u3001\u4f7f\u7528\u7684\u6743\u5229\u3002",
        0.6666155457496643
    ],
    [
        "**\u7b2c\u4e09\u767e\u4e03\u5341\u516d\u6761** \u5efa\u8bbe\u7528\u5730\u4f7f\u7528\u6743\u6d88\u706d\u7684\uff0c\u767b\u8bb0\u673a\u6784\u5e94\u5f53\u4f9d\u6cd5\u529e\u7406\u6ce8\u9500\u767b\u8bb0\u3002\n\u5efa\u8bbe\u7528\u5730\u4f7f\u7528\u6743\u4eba\u5e94\u5f53\u6309\u7167\u7ea6\u5b9a\u5411\u571f\u5730\u6240\u6709\u6743\u4eba\u652f\u4ed8\u51fa\u8ba9\u91d1\u3002",
        0.6666155457496643
    ],
    [
        "**\u7b2c\u4e09\u767e\u4e94\u5341\u4e94\u6761** \u7528\u76ca\u7269\u6743\u4eba\u884c\u4f7f\u6743\u5229\uff0c\u4e0d\u5f97\u635f\u5bb3\u6240\u

In [18]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [19]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的英文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [20]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的英文翻译，并用 <translated>和</translated> 标签标注。\n<context>\n**第三百八十六条** 居住权人有权对居住权的住宅进行维修、养护，但不得改变住宅的用途。\n居住权人对居住权的住宅享有占有、使用的权利。\n**第三百七十六条** 建设用地使用权消灭的，登记机构应当依法办理注销登记。\n建设用地使用权人应当按照约定向土地所有权人支付出让金。\n**第三百五十五条** 用益物权人行使权利，不得损害所有权人的合法权益。\n用益物权人不得转让、出租、担保其用益物权。法律另有规定或者当事人另有约定的除外。\n</context>\n<question>\n登记机构应履行那些职责?\n</question>\n<translated>\n</translated>\n'

In [21]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文，登记机构应履行的职责是：在建设用地使用权消灭时，依法办理注销登记。这一信息来源于《第三百七十六条》的内容。

<translated>
According to the provided context, the duties that the registration authority should perform are: when the right to use construction land is extinguished, the registration authority shall handle the cancellation of registration in accordance with the law. This information is derived from Article 376.
</translated>
